Module 2, Session 5- Minimax and Evaluation Functions
Objective- To understand the recursive logic of the Minimax algorithm and design an evaluation function for Tic-Tac-Toe.

Exercise 1
Trace the Minimax Algorithm (Conceptual)

Tree structure (levels and terminal utilities):

Level 0 (MAX): A

Level 1 (MIN): B, C, D  (children of A)

Level 2 (MAX):
- B -> E, F
- C -> G, H, I
- D -> J, K

Level 3 (Terminal values):
E = 3, F = 5, G = 6, H = 9, I = 7, J = 0, K = 1

Task:
1. Compute values for B, C, D (MIN nodes).
2. Compute value for root A (MAX).
3. Decide optimal first move from A: to B, C or D.
(You can do this on paper; below we also include a small Python check.)

In [1]:
terminal = {
    'E': 3, 'F': 5,
    'G': 6, 'H': 9, 'I': 7,
    'J': 0, 'K': 1
}

tree = {
    'B': ['E', 'F'],
    'C': ['G', 'H', 'I'],
    'D': ['J', 'K'],
    'A': ['B', 'C', 'D']
}

def minimax(node, maximizing):
    if node in terminal:
        return terminal[node]
    children = tree.get(node, [])
    if maximizing:
        best = float('-inf')
        for c in children:
            val = minimax(c, False)
            if val > best:
                best = val
        return best
    else:
        best = float('inf')
        for c in children:
            val = minimax(c, True)
            if val < best:
                best = val
        return best

val_B = minimax('B', False)
val_C = minimax('C', False)
val_D = minimax('D', False)
val_A = minimax('A', True)

print("Values:")
print("B =", val_B)
print("C =", val_C)
print("D =", val_D)
print("A =", val_A)

values_children = {'B': val_B, 'C': val_C, 'D': val_D}
best_move = max(values_children.items(), key=lambda x: x[1])
print("\nOptimal first move for MAX (A): go to", best_move[0], "with value", best_move[1])

Values:
B = 3
C = 6
D = 0
A = 6

Optimal first move for MAX (A): go to C with value 6


Exercise 1
Answers

- Values of MIN nodes:
  - B = min(E,F) = min(3,5) =3
  - C = min(G,H,I) = min(6,9,7) =6
  - D = min(J,K) = min(0,1) =0

- Root A (MAX): A = max(B,C,D) = max(3,6,0) =6
- Optimal first move for MAX: A → C (because C has value 6)

Exercise 2
Designing a Utility / Evaluation Function for Tic-Tac-Toe

Goal- create a fast evaluation(state) that scores a non-terminal board for X (MAX).
Hints / features to count:
- Two-in-a-row with an open cell (immediate threat/opportunity) — very important.
- One-in-a-row with two empties — small benefit.
- Control of the center — small bonus.
- Corner occupation — small bonus.
- Forks (two simultaneous threats) — big bonus.

In [2]:
def lines_indices():
    return [
        (0,1,2), (3,4,5), (6,7,8),
        (0,3,6), (1,4,7), (2,5,8),
        (0,4,8), (2,4,6)    
    ]

def eval_state(board):
    """Return score positive if good for X, negative if good for O."""
    two_in_row_weight = 100
    one_in_row_weight = 10
    center_weight = 15
    corner_weight = 5
    fork_weight = 200
    
    score = 0
    lines = lines_indices()
    
    x_two = 0
    o_two = 0
    x_one = 0
    o_one = 0
    
    for (i,j,k) in lines:
        trio = [board[i], board[j], board[k]]
        x_count = trio.count('X')
        o_count = trio.count('O')
        empty_count = trio.count(' ')
        if x_count == 2 and empty_count == 1:
            x_two += 1
        if o_count == 2 and empty_count == 1:
            o_two += 1
        if x_count == 1 and empty_count == 2:
            x_one += 1
        if o_count == 1 and empty_count == 2:
            o_one += 1
    
    score += two_in_row_weight * (x_two - o_two)
    score += one_in_row_weight * (x_one - o_one)
    
    if board[4] == 'X':
        score += center_weight
    elif board[4] == 'O':
        score -= center_weight
    
    corners = [0,2,6,8]
    score += corner_weight * sum(1 for c in corners if board[c]=='X')
    score -= corner_weight * sum(1 for c in corners if board[c]=='O')
    
    def count_potential_forks(player):
        forks = 0
        for idx in range(9):
            if board[idx] != ' ':
                continue
            board[idx] = player
            two_count = 0
            for (i,j,k) in lines:
                trio = [board[i], board[j], board[k]]
                if trio.count(player) == 2 and trio.count(' ') == 1:
                    two_count += 1
            if two_count >= 2:
                forks += 1
            board[idx] = ' '
        return forks
    
    x_forks = count_potential_forks('X')
    o_forks = count_potential_forks('O')
    score += fork_weight * (x_forks - o_forks)
    
    return score

empty_board = [' ']*9
board_example = [
    'X', 'X', ' ',
    ' ', 'O', ' ',
    ' ', ' ', 'O'
]

print("Empty board evaluation:", eval_state(empty_board))
print("Example board evaluation (X advantage):", eval_state(board_example))

board_example2 = [
    'O','O',' ',
    'X','X',' ',
    ' ',' ',' '
]
print("Example 2 evaluation:", eval_state(board_example2))

Empty board evaluation: 0
Example board evaluation (X advantage): -145
Example 2 evaluation: 420


What to include in submission

- For Exercise 1: show manual Minimax steps (B=3, C=6, D=0, A=6) and state optimal move A→C.
- For Exercise 2: include the evaluation function description and show a couple of example board evaluations (we included code tests).
- Save the notebook and upload to GitHub (or attach file to teacher).